### LP for Gaussian marginals X and Y and Gaussian spread X-Y
We demonstrate that these three 1D objects do not uniquely define the 2D joint density by finding extremal solutions for options on the index X+Y

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import cvxpy as cp
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt

from scipy import stats 
from scipy import interpolate
from sprd import grids, opt

In [3]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision = 4)
%matplotlib widget

In [4]:
# do we want to save figures for notes/slides
save_figures = False

Set up densities of X,Y and X-Y

In [5]:

# set up the grid for X and Y
npts = 101
fwd_for_grid = 0.0
std_for_grid = 1.0
n_std = 3
xgrid = grids.gen_grid(npts, fwd = fwd_for_grid, stdev=std_for_grid, nstd = n_std)
sx,sy = np.meshgrid(xgrid,xgrid,indexing = 'ij')

In [ ]:
# Set up marginal densities and the spread density to be Gaussian

x0 = 0
y0 = 0
xvol = 1.0
yvol = 1.0
xycor = 0.7
xdens,ydens,sdens,sgrid = grids.gen_gauss_densities(xgrid, x0,y0,xvol, yvol, xycor)

# check they are indeed densities
sum(xdens), sum(ydens), sum(sdens)

In [8]:
# Plot marginals if needed
test_plots = False
if test_plots:
    plt.figure()
    plt.plot(xgrid,xdens)
    plt.plot(xgrid,ydens)
    plt.title('X and Y densities')
    plt.show()

    plt.figure()
    plt.plot(sgrid,sdens)
    plt.title('X-Y density')
    plt.show()

         

Set up and solve LP for the min/max value of an option on index X+Y while keeping densities of X,Y and X-Y locked

In [9]:
def setup_and_solve_LP(type = 'min', method = 'ip', smooth_w = 1e2):
    '''
    This is the function to set up and solve the LP, either for type = 'min' or type = 'max'
        type: min or max
        method: ip or simplex
        smooth_w: smoothing weight for wquadratic optimization
    '''

    is_simplex = method != 'ip'

    # tolerance; equality makes this hard
    eps = 1e-6

    # 2d density to be solved for
    x = cp.Variable((npts,npts))

    # marginal and spread constraints
    constraints = opt.get_spread_constraints(x, xdens, ydens, sdens, eps = eps)

    # smoothing part of the objective
    smooth_obj = smooth_w * cp.sum(cp.diff(x)**2)


    # Set up the problem to solve
    if type == 'min':
        
        if is_simplex:
            obj = cp.Minimize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0) )
        else:
            obj = cp.Minimize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0) + smooth_obj)
    else: # 'max'

        if is_simplex:
            obj = cp.Maximize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0) )
        else:
            obj = cp.Maximize(opt.get_objf_idx(x, xgrid, idx_strike=x0+y0) - smooth_obj)
    prob = cp.Problem(obj, constraints)

    # solve

    if is_simplex:
        _ = prob.solve( solver = 'SCIPY', scipy_options = {'method':'highs'})
    else:
        _ = prob.solve()
    
    # Print status and the optimal value achieved
    print(f'Problem of type {type}, smoothing = {(not is_simplex) or (smooth_w <= 1e-6)}, status = {prob.status}, value = {prob.value:.4f}')

    # return the solution
    return x.value


In [ ]:
# Get the solutions for both min and max problems
pij_min = setup_and_solve_LP(type = 'min')
pij_max = setup_and_solve_LP(type = 'max')
pij_min_sp = setup_and_solve_LP(type = 'min', method='simplex')
pij_max_sp = setup_and_solve_LP(type = 'max', method = 'simplex')

In [11]:
test_plots = False
if test_plots:
    plt.figure()
    plt.plot(sgrid, sdens, label = 'true spread density')
    plt.plot(sgrid, grids.direct_diag_matrix_sums_basic(pij_min), 'o', label = 'fitted spread density for min')
    plt.plot(sgrid, grids.direct_diag_matrix_sums_basic(pij_max), '.', color = 'black', label = 'fitted spread density for max')
    plt.legend(loc = 'best')
    plt.show()


In [ ]:
# Plot the density of the index X+Y under two solutions
fig, ax1 = plt.subplots()
ax1.plot(sgrid, grids.reverse_diag_matrix_sums_basic(pij_min_sp), 
         label = 'Min index option value')

ax2 = ax1.twinx()
ax2.plot(sgrid, grids.reverse_diag_matrix_sums_basic(pij_max_sp), 
         color = 'orange', label = 'Max index option value')

plt.title('Density of the index X+Y, no smoothing')
ax1.legend(loc = 'upper left')
ax2.legend(loc = 'upper right')
plt.show()

if save_figures:
    fname = './figs/index_dens_for_min_max_opt_val__no_smooth_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

In [ ]:
# Plot the density of the index X+Y under two solutions
fig, ax1 = plt.subplots()
ax1.plot(sgrid, grids.reverse_diag_matrix_sums_basic(pij_min), 
         label = 'Min index option value')

ax2 = ax1.twinx()
ax2.plot(sgrid, grids.reverse_diag_matrix_sums_basic(pij_max), 
         color = 'orange', label = 'Max index option value')

plt.title('Density of the index X+Y')
ax1.legend(loc = 'upper left')
ax2.legend(loc = 'upper right')
plt.show()

if save_figures:
    fname = './figs/index_dens_for_min_max_opt_val_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

In [ ]:
# double_check our solution satisfies constraints
pij = pij_min # pij_max
print(np.sum(np.abs(pij.sum(axis=0) - xdens)) < 1e-5*npts)
print(np.sum(np.abs(pij.sum(axis=1) - ydens)) < 1e-5*npts)
print(np.sum(np.abs(grids.direct_diag_matrix_sums_basic(pij) - sdens)) < 1e-5*npts)

In [ ]:
step = 5
fig = plt.figure()
plt.plot(xgrid, pij_min[:,::step])
plt.title('Selected slices of the joint density for min index option value')

plt.show()
if save_figures:
    fname = './figs/joint_dens_slices_for_min_opt_val_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

In [ ]:
step = 5
fig = plt.figure()
plt.plot(xgrid, pij_max[:,::step])
plt.title('Selected slices of the joint density for max index option value')

plt.show()
if save_figures:
    fname = './figs/joint_dens_slices_for_max_opt_val_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(sx, sy, pij_min, alpha=0.7, cmap = 'YlGnBu')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Joint density for min index option value')
plt.legend(loc = 'best')
plt.show()
if save_figures:
    fname = './figs/joint_dens_2D_for_min_opt_val_01.pdf'
    fig.savefig(fname, bbox_inches='tight')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(sx, sy, pij_max, alpha=0.7, cmap = 'YlGnBu')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Joint density for max index option value')
plt.legend(loc = 'best')
plt.show()
if save_figures:
    fname = './figs/joint_dens_2D_for_max_opt_val_01.pdf'
    fig.savefig(fname, bbox_inches='tight')